In [17]:
import pandas as pd

In [18]:
weather_data_file = '../data/NOAA_weather_data/2005-2015_california_only_all.csv'
fire_data_file = '../data/USDA_wildfire_data/regridded_california_fires.csv'
training_data_file = '../data/training_data/2005-2015_training_data.csv'

# Parallelization options
N_PROCESSES = 15
JOBS_PER_PROCESS = 1

In [19]:
weather = pd.read_csv(weather_data_file)
weather['time'] = pd.to_datetime(weather['time'])
weather['date'] = weather['time'].dt.date

fires = pd.read_csv(fire_data_file)
fires['ignition'] = 1

In [20]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11978560 entries, 0 to 11978559
Data columns (total 14 columns):
time        datetime64[ns]
lat         float64
lon         float64
air.sfc     float64
air.2m      float64
apcp        float64
crain       float64
rhum.2m     float64
dpt.2m      float64
pres.sfc    float64
uwnd.10m    float64
vwnd.10m    float64
veg         float64
date        object
dtypes: datetime64[ns](1), float64(12), object(1)
memory usage: 1.2+ GB


In [21]:
fires.head()

,lat,lon,date,time,size,size_class,ignition
0,39.93427,-121.1598,2005-02-02,1300.0,0.10,A,1
1,38.85770,-120.5208,2004-05-12,845.0,0.25,A,1
2,39.13992,-120.5874,2004-05-31,1921.0,0.10,A,1
3,38.62627,-120.0940,2004-06-28,1600.0,0.10,A,1
4,38.62627,-120.0940,2004-06-28,1600.0,0.10,A,1


In [22]:
fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189550 entries, 0 to 189549
Data columns (total 7 columns):
lat           189550 non-null float64
lon           189550 non-null float64
date          189550 non-null object
time          110217 non-null float64
size          189550 non-null float64
size_class    189550 non-null object
ignition      189550 non-null int64
dtypes: float64(4), int64(1), object(2)
memory usage: 10.1+ MB


In [23]:
training_data = pd.merge(weather, fires, on=['lat', 'lon', 'date'], how='outer')

In [24]:
training_data.head()

,time_x,lat,lon,air.sfc,air.2m,apcp,crain,rhum.2m,dpt.2m,pres.sfc,uwnd.10m,vwnd.10m,veg,date,time_y,size,size_class,ignition
0,2005-01-01 00:00:00,40.29749,-124.3408,284.87308,283.24860,1.406250,1.0,77.472340,279.43463,98986.180,5.984521,3.811233,0.0,2005-01-01,NaN,NaN,NaN,NaN
1,2005-01-01 03:00:00,40.29749,-124.3408,284.71484,282.99774,0.781250,1.0,78.829380,279.48016,99001.150,3.470335,2.581940,0.0,2005-01-01,NaN,NaN,NaN,NaN
2,2005-01-01 06:00:00,40.29749,-124.3408,284.62878,282.55124,0.234375,1.0,77.277170,278.75513,99287.914,2.909558,1.104767,0.0,2005-01-01,NaN,NaN,NaN,NaN
3,2005-01-01 09:00:00,40.29749,-124.3408,284.64642,282.59882,1.875000,1.0,79.537100,279.22357,99256.030,2.127604,4.751480,0.0,2005-01-01,NaN,NaN,NaN,NaN
4,2005-01-01 12:00:00,40.29749,-124.3408,284.74457,282.18732,5.335938,1.0,80.818474,279.02075,99279.950,2.699432,6.547989,0.0,2005-01-01,NaN,NaN,NaN,NaN


In [25]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12168110 entries, 0 to 12168109
Data columns (total 18 columns):
time_x        datetime64[ns]
lat           float64
lon           float64
air.sfc       float64
air.2m        float64
apcp          float64
crain         float64
rhum.2m       float64
dpt.2m        float64
pres.sfc      float64
uwnd.10m      float64
vwnd.10m      float64
veg           float64
date          object
time_y        float64
size          float64
size_class    object
ignition      float64
dtypes: datetime64[ns](1), float64(15), object(2)
memory usage: 1.7+ GB


In [26]:
training_data.rename(columns={'time_x':'weather_bin_time','time_y':'fire_discovery_time'}, inplace=True)
training_data.drop(['date'], axis=1, inplace=True)
training_data.head()

,weather_bin_time,lat,lon,air.sfc,air.2m,apcp,crain,rhum.2m,dpt.2m,pres.sfc,uwnd.10m,vwnd.10m,veg,fire_discovery_time,size,size_class,ignition
0,2005-01-01 00:00:00,40.29749,-124.3408,284.87308,283.24860,1.406250,1.0,77.472340,279.43463,98986.180,5.984521,3.811233,0.0,NaN,NaN,NaN,NaN
1,2005-01-01 03:00:00,40.29749,-124.3408,284.71484,282.99774,0.781250,1.0,78.829380,279.48016,99001.150,3.470335,2.581940,0.0,NaN,NaN,NaN,NaN
2,2005-01-01 06:00:00,40.29749,-124.3408,284.62878,282.55124,0.234375,1.0,77.277170,278.75513,99287.914,2.909558,1.104767,0.0,NaN,NaN,NaN,NaN
3,2005-01-01 09:00:00,40.29749,-124.3408,284.64642,282.59882,1.875000,1.0,79.537100,279.22357,99256.030,2.127604,4.751480,0.0,NaN,NaN,NaN,NaN
4,2005-01-01 12:00:00,40.29749,-124.3408,284.74457,282.18732,5.335938,1.0,80.818474,279.02075,99279.950,2.699432,6.547989,0.0,NaN,NaN,NaN,NaN


In [27]:
training_data.describe()

,lat,lon,air.sfc,air.2m,apcp,crain,rhum.2m,dpt.2m,pres.sfc,uwnd.10m,vwnd.10m,veg,fire_discovery_time,size,ignition
count,1.216811e+07,1.216811e+07,1.197856e+07,1.197856e+07,1.197856e+07,1.197856e+07,1.197856e+07,1.197856e+07,1.197856e+07,1.197856e+07,1.197856e+07,1.197856e+07,110217.000000,189550.000000,189550.0
mean,3.712547e+01,-1.195300e+02,2.889527e+02,2.879476e+02,1.620509e-01,5.757771e-02,4.786626e+01,2.741508e+02,9.153190e+04,1.059241e+00,1.075154e-01,3.063439e+01,1383.752742,67.242725,1.0
std,2.556212e+00,2.502004e+00,1.392015e+01,1.023177e+01,9.353815e-01,2.329432e-01,2.640591e+01,7.587456e+00,6.966894e+03,2.732453e+00,3.306149e+00,2.275454e+01,490.324275,2029.960435,0.0
min,3.268389e+01,-1.243408e+02,2.422595e+02,2.465075e+02,0.000000e+00,0.000000e+00,1.000000e+00,1.921768e+02,6.624355e+04,-2.030144e+01,-2.000592e+01,0.000000e+00,0.000000,0.001000,1.0
25%,3.499029e+01,-1.214709e+02,2.788525e+02,2.807825e+02,0.000000e+00,0.000000e+00,2.521898e+01,2.691620e+02,8.715947e+04,-7.443695e-01,-2.100845e+00,8.200000e+00,1115.000000,0.100000,1.0
50%,3.683767e+01,-1.199051e+02,2.865459e+02,2.869761e+02,0.000000e+00,0.000000e+00,4.379562e+01,2.746586e+02,9.232952e+04,1.038500e+00,6.468201e-02,3.055000e+01,1428.000000,0.250000,1.0
75%,3.925970e+01,-1.174117e+02,2.974096e+02,2.947501e+02,0.000000e+00,0.000000e+00,6.971560e+01,2.797777e+02,9.708650e+04,2.748856e+00,2.251846e+00,4.360000e+01,1715.000000,1.000000,1.0
max,4.197182e+01,-1.143645e+02,3.384141e+02,3.229702e+02,4.148438e+01,1.000000e+00,1.000312e+02,2.993494e+02,1.035852e+05,2.156605e+01,2.987953e+01,8.370000e+01,2359.000000,315578.800000,1.0


In [28]:
training_data.to_csv(training_data_file, index=False)